In [3]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline

In [4]:
def show_image(fname, path='../99_image/', size=(20,20)):
    """读取*.png图片并显示"""
    plt.figure(figsize=size)
    plt.imshow(plt.imread(path+fname))
    plt.axis('off')
    plt.show()

In [5]:
def show_describe(data):
    """输出数据的基本信息"""
    _mean = 0.0 if data.mean() < 1e-11 else  data.mean()
    _std = data.std()
    _min = data.min()
    _max = data.max()
    _norm_1 = np.linalg.norm(data, ord=1)
    _norm_2 = np.linalg.norm(data, ord=2)
    if 1-_norm_2 < 1e-11: _norm_2 = 1.0
    print('mean={0}, std={1},'.format(_mean, _std,))
    print('min={0}, max={1},'.format(_min, _max,))
    print('l1_norm={0}, l2_norm={1}'.format(_norm_1, _norm_2))    

# 无监督

我们可以怎样发现一个数据集的底层结构？我们可以怎样最有用地对其进行归纳和分组？我们可以怎样以一种压缩格式有效地表征数据？这都是无监督学习的目标，之所以称之为「无监督」，是因为这是从无标签的数据开始学习的。

我们将在这里探索的两种无监督学习任务是：   
* 将数据按相似度聚类（clustering）成不同的分组；(K 均值聚类、层次聚类)
* 降维（reducing dimensionality）以便在保留数据结构和有用性的同时对数据进行压缩。(主成分分析（PCA）)

In [6]:
from sklearn.datasets import load_iris
#导入IRIS数据集
iris = load_iris()
#特征矩阵
print(iris.data[:5])
#目标向量
print(iris.target[:5])
#特征名称
print(iris.feature_names)

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]
[0 0 0 0 0]
['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']


In [7]:
iris_number = pd.DataFrame(iris.data, 
                           columns=iris.feature_names, )
iris_number.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


## 5.1 聚类
聚类的目标是为数据点分组，使得不同聚类中的数据点是不相似的，同一聚类中的数据点则是类似的。

### 5.1.1 K 均值聚类
使用 K 均值聚类，我们希望将我们的数据点聚类为 K 组。K 更大时，创造的分组就更小，就有更多粒度；K 更小时，则分组就更大，粒度更少。

该算法的输出是一组「标签」，这些标签将每个数据点都分配到了 K 组中的一组。在 K 均值聚类中，这些组的定义方式是为每个组创造一个重心（centroid）。这些重心就像是聚类的心脏，它们可以「捕获」离自己最近的点并将其加入到自己的聚类中。

你可以把这些重心看作是派对上成为关注焦点的人，他们就像是有磁性一样。如果只有一个这样的人，每个人都会围绕在他周围；如果有很多这样的人，就会形成很多更小一点的活动中心。

K 均值聚类的步骤如下：

* 定义 K 个重心。一开始这些重心是随机的（也有一些更加有效的用于初始化重心的算法）

* 寻找最近的重心并且更新聚类分配。将每个数据点都分配给这 K 个聚类中的一个。每个数据点都被分配给离它们最近的重心的聚类。这里的「接近程度」的度量是一个超参数——通常是欧几里得距离（Euclidean distance）。

* 将重心移动到它们的聚类的中心。每个聚类的重心的新位置是通过计算该聚类中所有数据点的平均位置得到的。

重复第 2 和 3 步，直到每次迭代时重心的位置不再显著变化（即直到该算法收敛）。

In [12]:
from sklearn.cluster import MiniBatchKMeans
from sklearn import metrics

#K-means,在训练集上训练
mb_kmeans = MiniBatchKMeans(n_clusters = 6)
mb_kmeans.fit(iris_number)

# 在训练集和测试集上测试
y_val_pred = mb_kmeans.predict(iris_number)
y_val_pred

array([5, 1, 1, 1, 5, 5, 1, 5, 1, 1, 5, 1, 1, 1, 5, 5, 5, 5, 5, 5, 5, 5,
       1, 5, 1, 1, 5, 5, 5, 1, 1, 5, 5, 5, 1, 1, 5, 1, 1, 5, 5, 1, 1, 5,
       5, 1, 5, 1, 5, 1, 0, 0, 0, 3, 0, 0, 0, 3, 0, 3, 3, 0, 3, 0, 3, 0,
       0, 3, 0, 3, 0, 3, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 0, 0, 0, 0, 0,
       3, 3, 3, 0, 3, 3, 3, 3, 3, 0, 3, 3, 4, 0, 2, 4, 4, 2, 3, 2, 4, 2,
       4, 4, 4, 0, 4, 4, 4, 2, 2, 0, 4, 0, 2, 0, 4, 2, 0, 0, 4, 2, 2, 2,
       4, 0, 4, 2, 4, 4, 0, 4, 4, 4, 0, 4, 4, 4, 0, 4, 4, 0])

In [21]:
# K值的评估标准
#常见的方法有轮廓系数Silhouette Coefficient和Calinski-Harabasz Index
#这两个分数值越大则聚类效果越好
#CH_score = metrics.calinski_harabaz_score(X_train,mb_kmeans.predict(X_train))
metrics.silhouette_score(iris_number, y_val_pred)

0.366230896649191

### 5.1.2  层次聚类
层次聚类类似于常规的聚类，只是你的目标是构建一个聚类的层次。如果你最终的聚类数量不确定，那这种方法会非常有用。比如说，假设要给 Etsy 或亚马逊等网络市场上的项目分组。在主页上，你只需要少量大组方便导航，但随着你的分类越来越特定，你需要的粒度水平也越来越大，即区别更加明显的项聚类。

在算法的输出方面，除了聚类分配，你也需要构建一个很好的树结构，以帮助你了解这些聚类之间的层次结构。然后你可以从这个树中选择你希望得到的聚类数量。

层次聚类的步骤如下：

* 首先从 N 个聚类开始，每个数据点一个聚类。

* 将彼此靠得最近的两个聚类融合为一个。现在你有 N-1 个聚类。

* 重新计算这些聚类之间的距离。有很多可以办到这件事的方法（参见这个教程了解更多细节：https://home.deib.polimi.it/matteucc/Clustering/tutorial_html/hierarchical.html）。其中一种方法（平均连接聚类，average-linkage clustering）是将两个聚类之间的距离看作是它们各自元素之间所有距离的平均。

* 重复第 2 和 3 步，直到你得到包含 N 个数据点的一个聚类。你就会得到如下图所示的树（也被称为树状图））。

* 选择一个聚类数量，然后在这个树状图中划一条水平线。比如说，如果你想要 K=2 个聚类，你应该在距离大约为 20000 的位置画一条水平线，你会得到一个包含数据点 8、9、11、16 的聚类和包含其它数据点的另一个聚类。一般而言，你得到的聚类的数量就是水平线与树状图中的竖直线的交叉点的数量。

In [29]:
from sklearn.cluster import DBSCAN
from sklearn import metrics
#DBSCAN,在训练集上训练

dbscan = DBSCAN(eps=0.3, min_samples=3)
y_train_pred = dbscan.fit_predict(iris_number)

y_train_pred

array([ 0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0, -1, -1, -1,
        0, -1,  0, -1,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0, -1, -1, -1,
        0,  0,  0,  0,  0,  0,  0, -1,  0,  0, -1,  0,  0,  0,  0,  0,  1,
       -1,  1,  4,  2, -1, -1, -1,  2, -1, -1, -1, -1,  3, -1,  2, -1,  4,
       -1,  4,  5, -1, -1,  3,  2,  2, -1, -1,  3, -1,  4,  4,  4, -1, -1,
       -1,  1, -1,  4,  4,  4,  3,  4, -1,  4,  4,  4,  2, -1,  4, -1,  6,
       -1,  7, -1, -1, -1, -1, -1, -1, -1, -1, -1,  6, -1, -1,  7, -1, -1,
       -1,  8, -1, -1,  5,  8, -1,  5,  5, -1, -1, -1, -1, -1, -1, -1, -1,
       -1,  7,  5, -1,  8, -1,  6,  8,  8, -1,  5, -1, -1,  5],
      dtype=int64)

In [30]:
# K值的评估标准
#常见的方法有轮廓系数Silhouette Coefficient和Calinski-Harabasz Index
#这两个分数值越大则聚类效果越好
#CH_score = metrics.calinski_harabaz_score(X_train,mb_kmeans.predict(X_train))
metrics.silhouette_score(iris_number, y_train_pred)

0.03189278933444991

In [101]:
print(X[0,:])
print(X_new[0,:])

[5.1 3.5 1.4 0.2]
[5.1 1.4 0.2]


## 5.2 降维
降维看上去很像压缩。这是为了在尽可能保存相关的结构的同时降低数据的复杂度。如果你有一张简单的 128×128×3 像素的图像（长×宽×RGB 值），那么数据就有 49152 维。如果你可以给这个图像空间降维，同时又不毁掉图像中太多有意义的内容，那么你就很好地执行了降维。

### 5.2.1 主成分分析（PCA）

我们可以修改空间的基础。现在想象有更高维度的空间，比如有 5 万维。你可以为这个空间选择一个基础，然后根据这个基础仅选择 200 个最重要的向量。这些基向量被称为主成分，而且你可以选择其中一个子集构成一个新空间，它的维度比原来的空间少，但又保留了尽可能多的数据复杂度。

要选择出最重要的主成分，我们需要检查这些数据的方差，并按这个指标给它们排序。

理解 PCA 的另一个思路是 PCA 将我们数据中存在的空间重映射成了一个更加紧凑的空间。这种变换后的维度比原来的维度更小。

仅需使用重映射空间的前几个维度，我们就可以开始理解这个数据集的组织结构。这就是降维的目的：减少复杂度（即这里的维度），同时保留结构（方差）。

In [34]:
from sklearn.decomposition import PCA

# 保留95%的信息
pca = PCA(n_components=0.95)
pca.fit(iris_number)

# 在训练集和测试集降维 
iris_number_pca = pca.transform(iris_number)
iris_number_pca[:5,:]

array([[-2.68420713,  0.32660731],
       [-2.71539062, -0.16955685],
       [-2.88981954, -0.13734561],
       [-2.7464372 , -0.31112432],
       [-2.72859298,  0.33392456]])

iris_number由原来的4维降为了2维

## 5.2.2 奇异值分解（SVD）
对矩阵进行分解, 可以分解为三个矩阵, 也可以分解为二个

# 5.3 树模型聚类

In [38]:
## Default XGBoost
import xgboost as xgb

y = iris.target
dtrain = xgb.DMatrix(iris_number, label=y)

In [45]:
params = {'max_depth':3,}
n_trees = 4
xgb1 = xgb.train(params, dtrain, n_trees,)

[07:36:35] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=3
[07:36:35] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 8 extra nodes, 0 pruned nodes, max_depth=3
[07:36:35] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 8 extra nodes, 0 pruned nodes, max_depth=3
[07:36:35] C:\dev\libs\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=3


In [46]:
pred_leaf = xgb1.predict(dtrain, pred_leaf=True)
pred_leaf[::10,:]

array([[1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [5, 5, 5, 5],
       [5, 5, 5, 5],
       [4, 6, 6, 6],
       [5, 5, 5, 5],
       [5, 5, 5, 5],
       [4, 8, 8, 4],
       [4, 8, 8, 4],
       [4, 8, 8, 4],
       [4, 8, 8, 4],
       [4, 8, 8, 4]])

4棵树, 各自有各自的叶子节点索引的输出   
叶子节点索引的最大值为$2^3=8$